In [26]:
import urllib.request
from PIL import Image
import requests
import json
import cv2
import base64
from pprint import pprint
from config import apiKey

# Generate Custom Meal Plan

## Get Recipe Ids

In [38]:
url = "https://api.spoonacular.com/recipes/complexSearch"
params = {
    "apiKey": apiKey,
    "diet": "gluten free",
    "cuisine": "italian",
    "excludeIngredients": "wheat",
    "number": 1
}
response1 = requests.get(url, params=params).json()
pprint(response1)

{'number': 1,
 'offset': 0,
 'results': [{'id': 773578,
              'image': 'https://spoonacular.com/recipeImages/773578-312x231.jpg',
              'imageType': 'jpg',
              'title': 'Pesto Salmon and Italian Veggies in Foil'}],
 'totalResults': 4788}


In [36]:
url = f"https://api.spoonacular.com/recipes/{response1['results'][0]['id']}/similar"
params = {
    "apiKey": apiKey,
    "number": 10
}
response2 = requests.get(url, params=params).json()
pprint(response2)

[{'id': 979822,
  'image': 'baked-spaghetti-and-meatballs-979822.jpg',
  'imageUrls': ['baked-spaghetti-and-meatballs-979822.jpg'],
  'readyInMinutes': 45,
  'servings': 6,
  'title': 'Baked Spaghetti and Meatballs'},
 {'id': 922154,
  'image': 'baked-spaghetti-meatballs-922154.jpg',
  'imageUrls': ['baked-spaghetti-meatballs-922154.jpg'],
  'readyInMinutes': 45,
  'servings': 4,
  'title': 'Baked Spaghetti & Meatballs'},
 {'id': 711741,
  'image': 'baked-spaghetti-and-meatballs-711741.jpg',
  'imageUrls': ['baked-spaghetti-and-meatballs-711741.jpg'],
  'readyInMinutes': 45,
  'servings': 4,
  'title': 'Baked Spaghetti and Meatballs'},
 {'id': 136475,
  'image': 'chicken-meatballs-for-spaghetti-and-meatballs-2-136475.jpg',
  'imageUrls': ['chicken-meatballs-for-spaghetti-and-meatballs-2-136475.jpg',
                'chicken_meatballs_for_spaghetti_and_meatballs-136475.jpg'],
  'readyInMinutes': 45,
  'servings': 4,
  'title': 'Chicken Meatballs For Spaghetti and Meatballs'},
 {'id': 29

In [3]:
base_url = "https://api.spoonacular.com/recipes/mealplans/generate?"

In [4]:
targetCalories = input("How many calories? ")
diet = input("Diet (vegetarian, gluten free, etc. ) " )
exclude = input("Allergic to anything? ")

## Check user response
if targetCalories == "n/a":
    targetCalories = ""
if diet == "n/a":
    diet = ""
if exclude == "n/a":
    exclude = ""

diet_list = ['gluten free', 'ketogenic', 'vegetarian', 'lacto-vegetarian',
             'ovo-vegetarian', 'vegan', 'pescetarian', 'paleo', 'primal',
             'whole30']
intolerances_list = ['dairy', 'egg', 'gluten', 'grain', 'peanut', 'seafood',
                  'sesame', 'shellfish', 'soy', 'sulfite', 'tree nut', 'wheat']
params = {
    "apiKey": apiKey,
    "timeFrame": "day",
    "targetCalories": targetCalories,
    "diet": diet,
    "exclude": exclude
}

How many calories? 1800
Diet (vegetarian, gluten free, etc. ) n/a
Allergic to anything? n/a


In [5]:
sparse_recipe = requests.get(base_url, params=params).json()

In [6]:
recipe_ids = [str(recipe['id']) for recipe in sparse_recipe['meals']]

## Get General Recipe Info

In [7]:
recipe_url = f"https://api.spoonacular.com/recipes/informationBulk?includeNutrition=true"
params = {
    "ids": ",".join(recipe_ids),
    'apiKey': apiKey
}

In [8]:
recipe_response = requests.get(recipe_url, params=params).json()

In [9]:
categories_to_keep = ['image', 'servings', 'readyInMinutes', 'sourceName',
                      'sourceUrl', 'aggregateLikes', 'pricePerServing',
                      'weightWatcherSmartPoints', 'spoonacularScore', 'title']

recipe_summary_info = {"breakfast": {}, "lunch": {}, "dinner": {}}
meals = [key for key in recipe_summary_info.keys()]
idx = 0

for recipe in recipe_response: 
    for key, value in recipe.items():
        if key in categories_to_keep:
            recipe_summary_info[meals[idx]][key] = value
    idx = idx + 1

## Get Recipe Ingredients and Instructions

In [10]:
recipe_instructions = {"breakfast": {}, "lunch": {}, "dinner": {}}
idx = 0

for recipe in recipe_response:
    recipe_instructions[meals[idx]] = recipe['instructions']
    idx = idx + 1

In [11]:
recipe_ingredients = {"breakfast": {}, "lunch": {}, "dinner": {}}
idx = 0

for recipe in recipe_response:
    ingredients = [f"{food['originalString']}\n" for food in recipe['extendedIngredients']]
    ingredients = "".join(ingredients)
    recipe_ingredients[meals[idx]] = ingredients
    idx = idx + 1

## Get Recipe Nutrition

In [18]:
recipe_calories = {"breakfast": {}, "lunch": {}, "dinner": {}}
idx = 0

for recipe in recipe_response:
    recipe_calories[meals[idx]] = recipe['nutrition']['caloricBreakdown']
    idx = idx+1

In [21]:
recipe_nutrients = {"breakfast": {}, "lunch": {}, "dinner": {}}
idx = 0

for recipe in recipe_response:
    recipe_nutrients[meals[idx]] = recipe['nutrition']['nutrients']